In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [4]:
import tensorflow_datasets as tfds # No Errors

In [5]:
imdb, info = tfds.load(
    "imdb_reviews",
    with_info = True, 
    as_supervised = True
)

In [6]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [9]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [10]:
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

In [11]:
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [13]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [14]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
tokenizer = Tokenizer(
    num_words = vocab_size,
    oov_token = oov_tok
)

In [18]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

In [19]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [20]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [21]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [23]:
model.compile(
    loss = 'binary_crossentropy', 
    optimizer = 'adam',
    metrics = ['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data = (testing_padded, testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 3s 3ms/step - loss: 0.6059 - accuracy: 0.6361 - val_loss: 0.3393 - val_accuracy: 0.8501
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2398 - accuracy: 0.9113 - val_loss: 0.3646 - val_accuracy: 0.8402
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0915 - accuracy: 0.9797 - val_loss: 0.4530 - val_accuracy: 0.8256
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0251 - accuracy: 0.9973 - val_loss: 0.5306 - val_accuracy: 0.8261
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0060 - accuracy: 0.9997 - val_loss: 0.5871 - val_accuracy: 0.8281
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.6412 - val_accuracy: 0.8284
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 9.2070e-04 - accuracy: 1.0000 - val_loss: 0.6822 - val_accuracy: 0.8285
Ep

In [27]:
e = model.layers[0]
weights = e.get_weights()[0]

print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [28]:
import io

out_v = io.open('vecs.tsv', 'w', encoding = 'utf-8')
out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')

In [30]:
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()